In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv(r"c:\Users\jayle\Downloads\insurance.csv")

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [4]:
df.describe()

,age,bmi,children,expenses
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.665471,1.094918,13270.422414
std,14.049960,6.098382,1.205493,12110.011240
min,18.000000,16.000000,0.000000,1121.870000
25%,27.000000,26.300000,0.000000,4740.287500
50%,39.000000,30.400000,1.000000,9382.030000
75%,51.000000,34.700000,2.000000,16639.915000
max,64.000000,53.100000,5.000000,63770.430000


In [5]:
# Plot Expense Boxplot
x0 = df['expenses']

# Create the figure
fig = go.Figure()

# Add trace
fig.add_trace(go.Box(x=x0,
                     name = 'charges',
                     marker_color='rgb(107,174,214)'))

fig.update_layout(title = 'Insurance Charges Boxplot')

fig.show()

### Simple Linear Regression

In [6]:
# Assinging X & y values
X = df['age'].values.reshape(-1, 1)
y = df['expenses'].values

In [7]:
# Printing value shapes
print(X.shape)
print(y.shape)

(1338, 1)
(1338,)


In [8]:
# Fit model
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [9]:
# Assign coefficient & intercept
coefficient = model.coef_[0]
intercept = model.intercept_

# Print values
print(f"Coefficient: {coefficient: .2f}")
print(f"Intercept: {intercept: .2f}")

Coefficient:  257.72
Intercept:  3165.89


In [10]:
# Calculate R2 value
r2 = r2_score(y, model.predict(X))
print(f"R_Squared: {r2: .4f}")

R_Squared:  0.0894


In [11]:
# Plot line of regression and scatter plot
x_range = np.linspace(X.min(), X.max(), 100)
y_range = model.predict(x_range.reshape(-1, 1))

# Create scatter plot
fig = px.scatter(df, x='age', y='expenses', opacity=0.65)
fig.update_traces(marker=dict(color='rgb(107,174,214)'))


fig.add_traces(go.Scatter(x=x_range,
                          y=y_range,
                          name='Regression Line',
                          line=dict(color='rgb(9,56,125)')))

# Update layout
fig.update_layout(
    title='Age vs Insurance Charges: Linear Regression',
    xaxis_title='Age',
    yaxis_title='Insurance Charges',
    height=600,
    width=800,
    showlegend=True
)

# Add equation and R-squared as annotations
equation = f'y = {coefficient:.2f}x + {intercept:.2f}'
r_squared_text = f'R² = {r2:.4f}'
fig.add_annotation(
    xref='paper', yref='paper',
    x=0.02, y=0.98,
    text=equation + '<br>' + r_squared_text,
    showarrow=False,
    font=dict(size=12),
    align='left'
)

fig.show()

### Multiple Linear Regression

In [12]:
# Import additional libraries for multiple regression
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [13]:
# Create dataframe copy
df2 = df.copy()

In [14]:
# Convert string values to binary values
df2['smoker'] = df2['smoker'].replace({'yes': 1, 'no': 0})

C:\Users\jayle\AppData\Local\Temp\ipykernel_14864\1823890725.py:2: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [15]:
df2.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [16]:
# Assign age, bmi, and smoker values to x1
X1 = df2[['age', 'bmi', 'smoker']]
y = df2['expenses']

In [17]:
# Add constant to independent variables
X1 = sm.add_constant(X1)

In [18]:
# Build model
model1 = sm.OLS(y, X1).fit()

In [19]:
print(model1.params)

const    -11679.047039
age         259.532924
bmi         322.691430
smoker    23822.606013
dtype: float64


In [20]:
# Make predictions based on the ols model
y_pred = model1.predict(X1)

# Calculate R-squared and Adjusted R-squared
r_squared = model1.rsquared
adj_r_squared = model1.rsquared_adj

# Calculate Mean Squared Error and Mean Absolute Error 
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)

# Print the results
print(f"R-squared: {r_squared:.4f}")
print(f"Adjusted R-squared: {adj_r_squared:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

R-squared: 0.7475
Adjusted R-squared: 0.7469
Mean Squared Error (MSE): 37003413.1911
Mean Absolute Error (MAE): 4216.5175


In [21]:
# Plot the regression models for the numerical variables

# Extracting coefficients and intercept from the model
coeff_age = model1.params['age']
coeff_bmi = model1.params['bmi']
coeff_smoker = model1.params['smoker']
intercept = model1.params['const']

# 1. Regression line for Age
x_age_range = np.linspace(X1['age'].min(), X1['age'].max(), 100)
y_age_range = intercept + coeff_age * x_age_range + coeff_bmi * X1['bmi'].mean() + coeff_smoker * X1['smoker'].mean()

# 2. Regression line for BMI
x_bmi_range = np.linspace(X1['bmi'].min(), X1['bmi'].max(), 100)
y_bmi_range = intercept + coeff_age * X1['age'].mean() + coeff_bmi * x_bmi_range + coeff_smoker * X1['smoker'].mean()

# Plot for Age vs Expenses
fig_age = px.scatter(df2, x='age', y='expenses', opacity=0.65)
fig_age.update_traces(marker=dict(color='rgb(107,174,214)'))

# Adding the regression line for Age
fig_age.add_traces(go.Scatter(x=x_age_range,
                              y=y_age_range,
                              name='Regression Line (Age)',
                              line=dict(color='rgb(9,56,125)')))

# Update layout for Age
fig_age.update_layout(
    title='Age vs Insurance Charges: Linear Regression',
    xaxis_title='Age',
    yaxis_title='Insurance Charges',
    height=600,
    width=800,
    showlegend=True
)

equation_age = f'y = {coeff_age:.2f}x + {intercept:.2f}'
r_squared_text = f'R² = {model1.rsquared:.4f}'
fig_age.add_annotation(
    xref='paper', yref='paper',
    x=0.02, y=0.98,
    text=equation_age + '<br>' + r_squared_text,
    showarrow=False,
    font=dict(size=12),
    align='left'
)

# Show the plot for Age
fig_age.show()


# Plot for BMI vs Expenses
fig_bmi = px.scatter(df2, x='bmi', y='expenses', opacity=0.65)
fig_bmi.update_traces(marker=dict(color='rgb(107,174,214)'))

# Adding the regression line for BMI
fig_bmi.add_traces(go.Scatter(x=x_bmi_range,
                              y=y_bmi_range,
                              name='Regression Line (BMI)',
                              line=dict(color='rgb(9,56,125)')))

# Update layout for BMI
fig_bmi.update_layout(
    title='BMI vs Insurance Charges: Linear Regression',
    xaxis_title='BMI',
    yaxis_title='Insurance Charges',
    height=600,
    width=800,
    showlegend=True
)

equation_bmi = f'y = {coeff_bmi:.2f}x + {intercept:.2f}'
r_squared_text = f'R² = {model1.rsquared:.4f}'
fig_bmi.add_annotation(
    xref='paper', yref='paper',
    x=0.02, y=0.98,
    text=equation_bmi + '<br>' + r_squared_text,
    showarrow=False,
    font=dict(size=12),
    align='left'
)

# Show the plot for BMI
fig_bmi.show()

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Check for collinearity
X_no_interaction = df2[['age', 'bmi', 'smoker']]
X_no_interaction = pd.get_dummies(X_no_interaction, drop_first=True)
X_no_interaction = sm.add_constant(X_no_interaction)

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["feature"] = X_no_interaction.columns
vif_data["VIF"] = [variance_inflation_factor(X_no_interaction.values, i) for i in range(X_no_interaction.shape[1])]

print(vif_data)

  feature        VIF
0   const  31.687225
1     age   1.012764
2     bmi   1.012146
3  smoker   1.000672


In [23]:
# Create interaction terms
df2['age_bmi_interaction'] = df2['age'] * df2['bmi']
df2['bmi_smoker_interaction'] = df2['bmi'] * df2['smoker']

# Fit model with interaction terms
X_interaction = df2[['age', 'bmi', 'smoker', 'age_bmi_interaction', 'bmi_smoker_interaction']]
X_interaction = sm.add_constant(X_interaction)

model_interaction = sm.OLS(y, X_interaction).fit()
print(model_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:               expenses   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     1363.
Date:                Mon, 07 Oct 2024   Prob (F-statistic):               0.00
Time:                        10:12:59   Log-Likelihood:                -13266.
No. Observations:                1338   AIC:                         2.654e+04
Df Residuals:                    1332   BIC:                         2.658e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     75

In [24]:
# Get predicted values from the model
y_pred = model_interaction.predict(X_interaction)

# R-squared and Adjusted R-squared
r_squared = model_interaction.rsquared
adj_r_squared = model_interaction.rsquared_adj

# Mean Squared Error and Mean Absolute Error
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)

# Print performance metrics
print(f"R-squared: {r_squared:.4f}")
print(f"Adjusted R-squared: {adj_r_squared:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

R-squared: 0.8365
Adjusted R-squared: 0.8359
Mean Squared Error (MSE): 23953838.3300
Mean Absolute Error (MAE): 2959.7526


In [25]:
# Analyze non-linear relationships between the dependent variables
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features for age and bmi
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(df2[['age', 'bmi']])

# Create a DataFrame for the polynomial features
poly_features = pd.DataFrame(X_poly, columns=['age', 'bmi', 'age^2', 'age*bmi', 'bmi^2'])

# Combine polynomial features with other variables
X_poly_full = pd.concat([poly_features, df2[['smoker']]], axis=1)
X_poly_full = pd.get_dummies(X_poly_full, drop_first=True)

# Add constant
X_poly_full = sm.add_constant(X_poly_full)

# Fit the polynomial regression model
model_poly = sm.OLS(y, X_poly_full).fit()

# Check the model summary
print(model_poly.summary())

                            OLS Regression Results                            
Dep. Variable:               expenses   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     664.8
Date:                Mon, 07 Oct 2024   Prob (F-statistic):               0.00
Time:                        10:12:59   Log-Likelihood:                -13551.
No. Observations:                1338   AIC:                         2.712e+04
Df Residuals:                    1331   BIC:                         2.715e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.479e+04   3977.715     -3.719      0.0

In [26]:
# Make predictions based on the polynomial regression model
y_pred = model_poly.predict(X_poly_full)

# Calculate R-squared and Adjusted R-squared
r_squared = model_poly.rsquared
adj_r_squared = model_poly.rsquared_adj

# Calculate Mean Squared Error and Mean Absolute Error 
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)

# Print the results
print(f"R-squared: {r_squared:.4f}")
print(f"Adjusted R-squared: {adj_r_squared:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

R-squared: 0.7498
Adjusted R-squared: 0.7487
Mean Squared Error (MSE): 36666102.6838
Mean Absolute Error (MAE): 4262.2636
